### Links

* https://wiseodd.github.io/techblog/2016/06/22/nn-optimization/ (Nasterov Momementum)
* https://wiseodd.github.io/techblog/2016/06/21/nn-sgd/ (Implementing Minibatch Gradient Descent)

In [ ]:
import _MEWtools as mt
import multiprocessing

## Get baseline time for CPU Intensive Workload

In [ ]:
iterations = 5

def slow_worker():
    for i in range(8000):
        square = i**i

    print('Done')

In [ ]:
import time

# t0 = time.time()

# for i in range(iterations):
#     slow_worker()

# t1 = time.time()

# total = t1-t0
# display(f'Total Execution Time: {total} seconds')

## Perform same operation with multiprocessing

In [ ]:
t0 = time.time()

pool = multiprocessing.Pool(iterations)
for i in range(iterations):
    pool.apply_async(slow_worker)
    
pool.close()
pool.join()

t1 = time.time()

total = t1-t0
display(f'Total Execution Time: {total} seconds')

## Load an interior structure from a config file and populate a MEWTools Satellite

In [ ]:
import utils

overrides = {
    'obliquity': 0,
    'obliquityPhase': 0,
    'obliquityPhaseRate': 0,
    'spinRate': 0,
    'nonSynchronusRotationRate': 0,
    'librationAmplitude': 0,
    'librationPhase': 0,
    'librationFrequency': 0
}

sample_sat = utils.import_structure('Sample', overrides)

In [ ]:
structures = utils.list_structures()
structures

In [ ]:
import sympy as sym
import numpy as np

r, θ, φ, t = sym.symbols('r θ φ t', real = True)
sample_sat.tt.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand()
sample_sat.tt.expand()

## Performance comparison of different evaluation methods

In [ ]:
import math

fm = sym.lambdify([t, φ, θ], sample_sat.PC1, modules = ["math", {"cot": math.atan}])
fn = sym.lambdify([t, φ, θ], sample_sat.PC1, modules = ["numpy", {"cot": np.arctan}])

# numpy
t0 = time.time()

for i in range(10000):
    fm(0, np.pi/2, np.pi/2)

t1 = time.time()

total = t1-t0
display(f'Total Execution Time for math lamdify: {total} seconds')

#math
t0 = time.time()

for i in range(10000):
    fn(0, np.pi/2, np.pi/2)

t1 = time.time()

total = t1-t0
display(f'Total Execution Time for numpy lamdify: {total} seconds')



# Expand
t0 = time.time()

for i in range(10000):
     sample_sat.PC1.subs(t, 0).subs(φ,np.pi/2).subs(θ,np.pi/2).expand()

t1 = time.time()

total = t1-t0
display(f'Total Execution Time for expand(): {total} seconds')


print(sample_sat.PC1.subs(t, 0).subs(φ,np.pi/2).subs(θ,np.pi/2).expand())
print(fm(0, np.pi/2, np.pi/2))
print(fn(0, np.pi/2, np.pi/2))

In [ ]:
# ttR = sym.re(sample_sat.tt.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand())
# ppR = sym.re(sample_sat.pp.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand())
# tpR = sym.re(sample_sat.tp.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand())

# import time

# diffs = []
# for i in range(1000):
#     t0 = time.time()

#     val1 = sample_sat.PC1.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand()
#     val2 = sample_sat.PC2.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand()
#     val3 = sample_sat.PCΨ.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand()
#     val4 = sample_sat.PCΨ2.subs(t,0).subs(φ,np.pi/4).subs(θ,np.pi/4).expand()

#     t1 = time.time()

#     total1 = t1-t0
# #     display(f'Total Execution Time: {total1} seconds')

#     import math
#     t0 = time.time()

#     pc1  = (1/2) * (ttR + ppR + math.sqrt(4*tpR**2 + (ttR-ppR)**2))
#     pc2  = (1/2) * (ttR + ppR - math.sqrt(4*tpR**2 + (ttR-ppR)**2))
#     pc3  = (1/2) * math.atan( (2*tpR)/(ttR-ppR))
#     pc4 = (1/2) * math.atan2((2*tpR),(ttR-ppR))

#     t1 = time.time()

#     total2 = t1-t0
# #     display(f'Total Optimized Execution Time: {total2} seconds')
#     diffs.append(total1 - total2)
    
# np.average(diffs)

## Checking degrees to radians performance with constant vs equation

In [ ]:
import time
import numpy as np

ITERATIONS = 1000000

t0 = time.time()

for i in range(ITERATIONS):
    rads = np.radians(60)

t1 = time.time()

total_np = t1-t0
display(f'Rads: {rads}')
display(f'Total Full Execution Time: {total_np} seconds')

# With constant
t0 = time.time()

rad_multiplier = np.pi / 180
for i in range(ITERATIONS):
    rads = 60 * rad_multiplier

t1 = time.time()

total_const = t1-t0
display(f'Rads: {rads}')
display(f'Total Execution Time: {total_const} seconds')

percent_faster = round(((total_np-total_const)/total_np) * 100, 2)
display(f'Using a constant is {percent_faster}% faster than using numpy.radians')

**Theta = Longitude**

**Phi = Latitude**

## Lets try to build a grid

In [ ]:
import pandas as pd
import numpy as np
import math
import sympy as sym
import multiprocessing, logging

logger = multiprocessing.log_to_stderr()
logger.setLevel(logging.INFO)

r, θ, φ, t = sym.symbols('r θ φ t', real = True)

t0 = time.time()

sat = utils.import_structure('Sample', overrides)

t1 = time.time()
total = (t1 - t0)
           
print(f'Load Structure time: {total}')

TIME_STEPS = 360
MIN_LAT = -75
MAX_LAT = 90
MIN_LON = 0
MAX_LON = 360
RAD_MULTIPLIER = np.pi / 180

data = []

principal1_func = sym.lambdify([t, φ, θ], sample_sat.PC1, modules = ["math", {"cot": math.atan}])
principal2_func = sym.lambdify([t, φ, θ], sample_sat.PC2, modules = ["math", {"cot": math.atan}])
principal_phi_func = sym.lambdify([t, φ, θ], sample_sat.PCΨ, modules = ["math", {"cot": math.atan}])
principal_phi2_func = sym.lambdify([t, φ, θ], sample_sat.PCΨ2, modules = ["math", {"cot": math.atan}])


def callback(stress_items):
    data.extend(stress_items)
    

def get_stress_for_lat(step, lat):
    results = []
#     lat_radians = lat * RAD_MULTIPLIER
    lat_radians = np.radians(lat)
    step_value = step / TIME_STEPS
    
    logger.info("ABOUT TO PERFORM CALCULATIONS")
    
    for lon in range(MIN_LON, MAX_LON + 1, 10):
        if (lat == 90 or lon == 0):
            continue
            
#         lon_radians = np.Radians(lon) * RAD_MULTIPLIER
        lon_radians = np.radians(lon)

        principal1 = principal1_func(step_value, lat_radians, lon_radians)
#         principal1 = sat.PC1.subs(t,step_value).subs(φ,lat_radians).subs(θ,lon_radians).expand()

        principal2 = principal2_func(step, lat_radians, lon_radians)
        principal_phi = principal_phi_func(step, lat_radians, lon_radians)
        principal_phi2 = principal_phi2_func(step, lat_radians, lon_radians)        

        max_stress = max(principal1, principal2)
        max_stress_orientation = principal_phi if max_stress == principal1 else principal_phi2
  
        
        results.append({
                'time_step': step,
                'latitude': lat,
                'longitude': lon,
                'principal1': principal1,
                'principal2': principal2,
                'principal_orientation': np.rad2deg(principal_phi),
                'principal_orientation2': np.rad2deg(principal_phi2),
                'max_stress': max_stress,
                'max_stress_orientation': np.rad2deg(max_stress_orientation)
            })
        
    return results

    
pool = multiprocessing.Pool()

t0 = time.time()

for step in range(TIME_STEPS):
    for lat in range(MIN_LAT, MAX_LAT + 1, 15):
        pool.apply_async(get_stress_for_lat, args = (step, lat, ), callback=callback)
#         data.extend(get_stress_for_lat(step, lat))
            
pool.close()
pool.join()
            
t1 = time.time()
total = (t1 - t0)
           
print(f'Calculation time: {total}')

df = pd.DataFrame(data)   
t2 = time.time()
print(f'DataFrame Creation time: {t2 - t1}')


In [ ]:
lat = 60
lon = 250

print(principal1_func(0, lat * RAD_MULTIPLIER, lon * RAD_MULTIPLIER))
print(principal2_func(0, lat * RAD_MULTIPLIER, lon * RAD_MULTIPLIER))
print(principal_phi_func(0, lat * RAD_MULTIPLIER, lon * RAD_MULTIPLIER))
print(principal_phi2_func(0, lat * RAD_MULTIPLIER, lon * RAD_MULTIPLIER))

In [ ]:
df.sort_values(['latitude', 'longitude', 'time_step'])

In [ ]:
import StressTools as tools
import utils
import time

t0 = time.time()

sat = utils.import_structure('Sample')

t1 = time.time()
total = (t1 - t0)
           
print(f'Load Structure time: {total}')


In [ ]:
t0 = time.time()

df = tools.build_stress_field(sat)

t1 = time.time()
total = (t1 - t0)
           
print(f'Build Stress Field time: {total}')

In [ ]:
data = df.loc[(df['latitude'] == 45) & (df['longitude'] == 60)] \
    .sort_values('time_step')

import matplotlib.pyplot as plt

plt.scatter(data['time_step'], data['max_stress'])
data

In [ ]:
tools.get_principal1